In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [ ]:
from pathlib import Path

from wikipedia_cleanup.data_filter import KeepAttributesDataFilter, generate_default_filters
from wikipedia_cleanup.predict import TrainAndPredictFramework
from wikipedia_cleanup.predictor import ZeroPredictor, OnePredictor, MeanPredictor, RandomPredictor
from wikipedia_cleanup.property_correlation import PropertyCorrelationPredictor

In [ ]:
n_files = 2
n_jobs = 4
input_path = Path("../../data/custom-format-default-filtered/")
#input_path = Path("/run/media/secret/manjaro-home/secret/mp-data/custom-format-default-filtered")
model = PropertyCorrelationPredictor(use_cache=True)
framework = TrainAndPredictFramework(model, group_key=['infobox_key', 'property_name'])

In [ ]:
framework.load_data(input_path, n_files, n_jobs)

In [ ]:
framework.fit_model()

In [ ]:
framework.test_model()

In [ ]:
n_files=8
n_jobs=4
input_path = Path("../../data/custom-format-default-filtered/")
keep_filter = KeepAttributesDataFilter(['infobox_key', 'property_name', 'page_id', 'value_valid_from'])

data = get_data(input_path,n_files=n_files, n_jobs=n_jobs, filters=[keep_filter])

In [ ]:
data['value_valid_from'] = data['value_valid_from'].dt.tz_localize(None)

In [ ]:
test_dates = pd.date_range(datetime(2015,1,1), datetime(2015,1,20) + timedelta(days=5))


In [ ]:
[date in data[data['page_id'] == 650095]['value_valid_from'] for date in test_dates]

In [ ]:
data[data['page_id'] == 650095]

In [ ]:
testset_start = datetime(2018,9,1)
testset_duration = 365
total_time_window = timedelta(testset_duration)#days
testset_end = testset_start + total_time_window
time_offset = timedelta(1)

In [ ]:
class ZeroPredictor:
    def predict_day(self, change_data, current_day):
        return False
    def predict_week(self, change_data, current_day):
        return self.predict_day(change_data, current_day)
    def predict_month(self, change_data, current_day):
        return self.predict_day(change_data, current_day)
    def predict_year(self, change_data, current_day):
        return self.predict_day(change_data, current_day)

class DummyPredictor(ZeroPredictor):
    def predict_day(self, change_data, current_day):
        pred = next_change(change_data)
        if pred is None:
            return False
        return pred - current_day <= timedelta(1)

In [ ]:
# number of changes in our testing period
((data['value_valid_from'] > testset_start) & (data['value_valid_from'] < (testset_start+total_time_window))).sum()

In [ ]:
class Predictor:
    @staticmethod
    def get_relevant_attributes():
        return
    
    def fit(self, train_data, last_day):
        pass
    
    def predict_day(self, data, current_day):
        return False
    
    def predict_week(self, data, current_day):
        return False

    def predict_month(self, data, current_day):
        return False

    def get_relevant_ids(self, identifier):
        return list(identifier)

In [ ]:
class TrainAndPredictFramework:
    
    def start_pipeline(self):
        
        

In [ ]:
data[data['value_valid_from'] < testset_start]

In [ ]:
#property_change_history = data.groupby(['infobox_key', 'property_name'])["value_valid_from"].agg(list)+
property_change_history = data.groupby(['page_id'])["value_valid_from"].agg(list)